In [0]:
%sql
--Query to find the total count of customers by state and country:
SELECT state, country, count(*) as total_count
FROM lending_loan_e2e.customer_details_external
GROUP BY state, country;

state,country,total_count
Michigan,USA,7286
Virginia,USA,7216
South Carolina,USA,7225
Utah,USA,7268
Maryland,USA,7179
New York,USA,7395
Wyoming,USA,7300
Acceptable,USA,7201
Arizona,USA,7288
Washington,USA,7151


####Query to find the count of customers having premium membership falling in different age buckets


In [0]:
%sql
SELECT country, 
       CASE 
           WHEN age BETWEEN 18 AND 25 THEN 'Youngsters'
           WHEN age BETWEEN 26 AND 35 THEN 'Working class'
           WHEN age BETWEEN 36 AND 45 THEN 'Middle Age'
           ELSE 'Senior Citizens'
       END as age_range,
       COUNT(*)
 FROM lending_loan_e2e.customer_details_external
WHERE premium_status = 'TRUE'
GROUP BY country, age_range;

country,age_range,count(1)
USA,Youngsters,7208
USA,Senior Citizens,40140
USA,Middle Age,8884
USA,Working class,8961


####Query to find the percentage of customers in each state that are premium customers, grouped by country


In [0]:
%sql
WITH customer_counts AS (
    SELECT country, state, COUNT(*) as total_customers
    FROM lending_loan_e2e.customer_details_external
    GROUP BY country, state
),
member_counts AS (
    SELECT country, state, COUNT(DISTINCT member_id) as total_members
    FROM lending_loan_e2e.customer_details_external
    WHERE member_id IS NOT NULL and  premium_status = 'TRUE'
    GROUP BY country, state
)
SELECT customer_counts.country, customer_counts.state, 
       ROUND(member_counts.total_members / customer_counts.total_customers * 100, 2) as member_percentage
FROM customer_counts
JOIN member_counts
ON customer_counts.country = member_counts.country AND customer_counts.state = member_counts.state;

country,state,member_percentage
USA,Utah,50.23
USA,Hawaii,49.56
USA,New Jersey,50.0
USA,New York,49.43
USA,Wyoming,49.59
USA,California,49.79
USA,Pennsylvania,49.88
USA,Illinois,49.58
USA,Washington,50.09
USA,Arizona,49.84


In [0]:
from pyspark.sql.functions import avg, count,col,when,countDistinct,round

In [0]:
customers_df = spark.read.parquet("/mnt/silver/lending_loan/customer_details")

####Query to find the number of customers with a premium status of "true" in each country, grouped by age range using pyspark dataframe:

In [0]:
customers_df.filter("premium_status = 'TRUE'" ) \
  .withColumn("age_range", when((col("age") >= 18) & (col("age") <= 25), "Youngsters")
                           .when((col("age") > 25) & (col("age") <= 35), "Working class")
                           .when((col("age") > 35) & (col("age") <= 45), "Middle Age")
                           .otherwise("Senior citizens")) \
  .groupBy("country", "age_range") \
  .agg(count("*")) \
  .show()

+-------+---------------+--------+
|country|      age_range|count(1)|
+-------+---------------+--------+
|    USA|     Youngsters|    7208|
|    USA|     Middle Age|    8884|
|    USA|  Working class|    8961|
|    USA|Senior citizens|   40140|
+-------+---------------+--------+



In [0]:
#Query to find the average age of customers by state and country using pyspark dataframe
customer_avg_age=customers_df.groupBy("state", "country").agg(avg("age"))

In [0]:
customer_avg_age.createOrReplaceTempView("customers_avg_age")

In [0]:
%sql
CREATE EXTERNAL TABLE lending_loan_e2e.customers_avg_age 
USING PARQUET
LOCATION '/mnt/silver/lending-loan/customer-transformations/customers_avg_age'
select * from customers_avg_age

In [0]:
%sql
SELECT * FROM lending_loan_e2e.customers_avg_age

state,country,avg(age)
Michigan,USA,54.04899807850673
Virginia,USA,53.9539911308204
South Carolina,USA,53.858269896193775
Utah,USA,54.34411117226197
Maryland,USA,53.78534614848865
New York,USA,53.87342799188641
Wyoming,USA,54.14917808219178
Acceptable,USA,54.041383141230384
Arizona,USA,54.37966520307354
Washington,USA,53.774716822821986


In [0]:
%sql
CREATE EXTERNAL TABLE lending_loan_e2e.customers_premium_status 
USING PARQUET
LOCATION '/mnt/datasetbigdata/processed-data/lending-loan/customer-transformations/customers_premium_status'

In [0]:
%sql

 WITH customer_counts AS (
    SELECT country, state, COUNT(*) as total_customers
    FROM lending_loan_e2e.customer_details_external
    GROUP BY country, state
),
member_counts AS (
    SELECT country, state, COUNT(DISTINCT member_id) as total_members
    FROM lending_loan_e2e.customer_details_external
    WHERE member_id IS NOT NULL and  premium_status = 'TRUE'
    GROUP BY country, state
)
SELECT customer_counts.country, customer_counts.state, 
       ROUND(member_counts.total_members / customer_counts.total_customers * 100, 2) as member_percentage
FROM customer_counts
JOIN member_counts
ON customer_counts.country = member_counts.country AND customer_counts.state = member_counts.state;

country,state,member_percentage
USA,Utah,50.23
USA,Hawaii,49.56
USA,New Jersey,50.0
USA,New York,49.43
USA,Wyoming,49.59
USA,California,49.79
USA,Pennsylvania,49.88
USA,Illinois,49.58
USA,Washington,50.09
USA,Arizona,49.84


In [0]:
customers_premium_status_percentage = spark.sql("select * from lending_loan_e2e.customers_premium_status")

In [0]:
display(customers_premium_status_percentage)

country,state,member_percentage
USA,Utah,50.23
USA,Hawaii,49.56
USA,New Jersey,50.0
USA,New York,49.43
USA,Wyoming,49.59
USA,California,49.79
USA,Pennsylvania,49.88
USA,Illinois,49.58
USA,Washington,50.09
USA,Arizona,49.84


In [0]:
customers_premium_status_percentage.write.options(header='True').mode("append").csv("/mnt/gold/lending_loan/customers_premium_status_percentage")

In [0]:
customer_premium_df = spark.read \
.option("header", True)  \
.csv("/mnt/gold/lending_loan/customers_premium_status_percentage")

In [0]:
display(customer_premium_df)

country,state,member_percentage
USA,Utah,50.23
USA,Hawaii,49.56
USA,New Jersey,50.0
USA,New York,49.43
USA,Wyoming,49.59
USA,California,49.79
USA,Pennsylvania,49.88
USA,Illinois,49.58
USA,Washington,50.09
USA,Arizona,49.84
